In [1]:
import numpy as np
from cnn_utils import *
from dlsuper.cnn import CNNLayer, PoolingLayer, CNNModel, CNNPipe
from dlsuper.nn.NNLayer import NNLinearLayer, NNActivator
from dlsuper.nn.NNPipe import NNPipe
from dlsuper.cnn import CNNModel

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()
X_train = X_train_orig/255.
X_train = X_train.transpose(1, 2, 3, 0)
X_test = X_test_orig/255.
X_test = X_test.transpose(1, 2, 3, 0)
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_train = Y_train.transpose(1, 0)
Y_test = convert_to_one_hot(Y_test_orig, 6).T
Y_test = Y_test.transpose(1, 0)

In [2]:
import math
def compute_samemode_pad(in_width, in_height, filter_size, strides):
    # 先确定输出维度，记住是上取整
    stride_width, stride_height = strides
    filter_width, filter_height = filter_size
    out_height = math.ceil(float(in_height) / float(stride_height))
    out_width  = math.ceil(float(in_width) / float(stride_width))

    # 上面的公式
    if (in_height % stride_height == 0):
        pad_along_height = max(filter_height - stride_height, 0)
    else:
        pad_along_height = max(filter_height - (in_height % stride_height), 0)
    if (in_width % stride_width == 0):
        pad_along_width = max(filter_width - stride_width, 0)
    else:
        pad_along_width = max(filter_width - (in_width % stride_width), 0)

    # 因为pad是在上下、左右四侧pad。所以当pi不为偶数时要分配下
    # 这里是当pi为奇数时，下侧比上侧多一，右侧比左侧多一。
    #  Note that this is different from existing libraries such as cuDNN and Caffe, which explicitly specify the number of padded pixels and always pad the same number of pixels on both sides.
    pad_top = pad_along_height // 2
    pad_bottom = pad_along_height - pad_top
    pad_left = pad_along_width // 2
    pad_right = pad_along_width - pad_left
    return (pad_left, pad_right, pad_top, pad_bottom)

In [ ]:
pad = compute_samemode_pad(64, 64, (4, 4), (1, 1))
cnn_layer1 = CNNLayer.CNNLayer(None, (4, 4, 3), 8, pad, (1, 1), "relu")

pad = compute_samemode_pad(64, 64, (8, 8), (8, 8))
pool_layer1 = PoolingLayer.PoolingLayer(cnn_layer1, (8, 8), pad, "max", (8, 8))
pool_layer1.layer = 1

pad = compute_samemode_pad(8, 8, (2, 2), (1, 1))
cnn_layer2 = CNNLayer.CNNLayer(pool_layer1, (2, 2, 8), 16, pad, (1, 1), "relu")

pad = compute_samemode_pad(8, 8, (4, 4), (4, 4))
pool_layer2 = PoolingLayer.PoolingLayer(cnn_layer2, (4, 4), pad, "max", (4, 4))
pool_layer2.layer = 2

cnn_pipe = CNNPipe.CNNPipe(cnn_layer1, pool_layer2)


liner_layer = NNLinearLayer(None, 64, 6)
outputLayer = NNActivator(liner_layer, 'softmax', 10) 
nn_pipe = NNPipe(liner_layer, outputLayer)

model = CNNModel.CNNModel(cnn_pipe, nn_pipe)
model.fit(X_train[:, :, :,0:128], Y_train[:, 0:128], learning_rate=0.1, iteration_count=200, lambd=0,
            use_mini_batch=True, mini_batch_size=32)

[0.16117012 0.18812745 0.13882606 0.1709351  0.14660072 0.19434055]
[0.15893104 0.18641108 0.14086814 0.17767776 0.14936681 0.18674517]
[0.16268629 0.18681247 0.13981959 0.17578017 0.14473531 0.19016617]
0.1875
iteration1    cost:57.58287344527255
[1.05295204e-228 2.26788323e-205 1.00000000e+000 6.44283078e-217
 0.00000000e+000 1.30167194e-118]
[2.50904037e-227 8.49019714e-210 1.00000000e+000 6.74487691e-214
 0.00000000e+000 7.49733259e-120]
[9.06805526e-225 1.26792996e-203 1.00000000e+000 6.82447674e-210
 0.00000000e+000 4.69308049e-113]
0.28125
iteration2    cost:829.0040279496944
[0.00000000e+000 0.00000000e+000 1.00000000e+000 0.00000000e+000
 0.00000000e+000 3.24406934e-265]
[0.00000000e+000 0.00000000e+000 1.00000000e+000 0.00000000e+000
 0.00000000e+000 4.88569408e-259]
[0.00000000e+000 0.00000000e+000 1.00000000e+000 0.00000000e+000
 0.00000000e+000 1.10306408e-252]
0.09375
iteration3    cost:1045.2659482843974
[0. 0. 1. 0. 0. 0.]
[0. 0. 1. 0. 0. 0.]
[0. 0. 1. 0. 0. 0.]
0.125
i